In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]

### 격자(매핑용)

In [6]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [7]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 요일별 유동인구

In [8]:
# 원도심 영역(도시재생 대상지역)
floating_population_df_day = pd.read_csv('SBJ_2309_001/4.청주시_요일별_유동인구.csv')
floating_population_df_day['STD_YM'] = floating_population_df_day['STD_YM'].astype(str)
floating_population_df_day.rename(columns={'STD_YM':'연도'}, inplace=True)
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
floating_population_df_day = geo_transform(floating_population_df_day)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# floating_population_df_day의 'geometry' 열을 사용하여 Point 객체를 필터링
floating_population_df_day = floating_population_df_day[floating_population_df_day['geometry'].within(polygon)]

# 열 이름 변경 과정
columns_to_rename = floating_population_df_day.columns[1:-3] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    day_part = column_name.split('_')[0] # 요일부분 추출
    new_column_names.append(day_part) # day_part에 추가
# 열 이름을 변경합니다.
floating_population_df_day.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_day

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,연도,mon,tue,wed,thu,fri,sat,sun,lon,lat,geometry
94061,202001,49.28,51.12,50.75,54.67,52.78,51.24,46.99,127.483602,36.631043,POINT (127.48360 36.63104)
94062,202001,112.92,114.62,115.31,121.40,121.01,123.04,114.80,127.483602,36.631494,POINT (127.48360 36.63149)
94063,202001,119.92,122.71,120.15,129.35,125.74,112.61,104.84,127.483602,36.631944,POINT (127.48360 36.63194)
94064,202001,82.00,85.41,84.07,90.94,87.51,82.13,71.36,127.483602,36.632395,POINT (127.48360 36.63240)
94366,202001,59.56,63.09,60.48,64.91,63.13,57.99,50.31,127.484162,36.630592,POINT (127.48416 36.63059)
...,...,...,...,...,...,...,...,...,...,...,...
4687534,202212,73.23,72.03,70.74,72.24,79.62,56.35,45.40,127.492551,36.631945,POINT (127.49255 36.63195)
4687535,202212,284.77,279.83,278.79,281.96,304.53,289.08,239.32,127.492550,36.632396,POINT (127.49255 36.63240)
4687536,202212,101.30,100.28,98.78,101.39,103.76,83.35,71.96,127.492550,36.632847,POINT (127.49255 36.63285)
4687958,202212,263.03,263.19,253.53,263.11,280.09,290.16,267.59,127.493110,36.628339,POINT (127.49311 36.62834)


#### 유동인구 HEATMAP의 시계열(요일) 변화 시각화

In [9]:
# 딕셔너리 생성 및 연도 & DataFrame 매핑
filtered_floating_population_day = {}
year_list = ['2020', '2021', '2022']

for year in year_list:
    year_df = pd.DataFrame({'연도' : [year]}) # 해당 연도 df 설정(접합용)
    year_tot = pd.DataFrame() # 해당 연도 df 설정(최종용)
    floating_population_df_day_year = floating_population_df_day[floating_population_df_day['연도'].str.startswith(year)] # 특정 연도로만 필터링
    for point in floating_population_df_day_year['geometry'].unique():
        point_df = pd.DataFrame({'lon' : [point.x], 'lat' : [point.y]}) # 해당 지점 df 설정(접합용)
        fpt_point_df = floating_population_df_day_year[floating_population_df_day_year['geometry'] == point]
        
        # 1월부터 12월 까지의 월별 유동인구 평균계산 및 연도, 지점 df와 접합 & 최종 df에 접합
        column_mean = fpt_point_df.iloc[:,1:8].mean()
        column_mean_df = pd.DataFrame(column_mean).transpose()
        column_mean_df = column_mean_df.round(2) # 소수점 2번째 자리까지만 표기
        column_mean_df = pd.concat([year_df, column_mean_df, point_df], axis=1)
        year_tot = pd.concat([year_tot, column_mean_df])
    else:
        # 유동인구 정규화 과정
        max_val = year_tot.iloc[:, 1:-2].values.max() # 유동인구의 최댓값
        year_tot.iloc[:, 1:-2] = (year_tot.iloc[:, 1:-2] / max_val) * 100 # 최댓값으로 나누어 100을 곱하여 정규화
        filtered_floating_population_day[year] = year_tot

#### 2020년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [10]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2020']['lat']
longitudes = filtered_floating_population_day['2020']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2020'].columns[1:-2]:
    volume = filtered_floating_population_day['2020'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2021년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [11]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2021']['lat']
longitudes = filtered_floating_population_day['2021']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2021'].columns[1:-2]:
    volume = filtered_floating_population_day['2021'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2022년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [12]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2022']['lat']
longitudes = filtered_floating_population_day['2022']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2022'].columns[1:-2]:
    volume = filtered_floating_population_day['2022'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 격자에 매핑 & 평일과 주말을 통합한 격자의 유동인구 시각화

In [13]:
floating_population_df_day

,연도,mon,tue,wed,thu,fri,sat,sun,lon,lat,geometry
94061,202001,49.28,51.12,50.75,54.67,52.78,51.24,46.99,127.483602,36.631043,POINT (127.48360 36.63104)
94062,202001,112.92,114.62,115.31,121.40,121.01,123.04,114.80,127.483602,36.631494,POINT (127.48360 36.63149)
94063,202001,119.92,122.71,120.15,129.35,125.74,112.61,104.84,127.483602,36.631944,POINT (127.48360 36.63194)
94064,202001,82.00,85.41,84.07,90.94,87.51,82.13,71.36,127.483602,36.632395,POINT (127.48360 36.63240)
94366,202001,59.56,63.09,60.48,64.91,63.13,57.99,50.31,127.484162,36.630592,POINT (127.48416 36.63059)
...,...,...,...,...,...,...,...,...,...,...,...
4687534,202212,73.23,72.03,70.74,72.24,79.62,56.35,45.40,127.492551,36.631945,POINT (127.49255 36.63195)
4687535,202212,284.77,279.83,278.79,281.96,304.53,289.08,239.32,127.492550,36.632396,POINT (127.49255 36.63240)
4687536,202212,101.30,100.28,98.78,101.39,103.76,83.35,71.96,127.492550,36.632847,POINT (127.49255 36.63285)
4687958,202212,263.03,263.19,253.53,263.11,280.09,290.16,267.59,127.493110,36.628339,POINT (127.49311 36.62834)


In [14]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_totday = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = floating_population_df_day[floating_population_df_day['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in floating_population_df_day['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:8].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_totday에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:8].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        column_means_2020_df = pd.concat([column_means_2020_df.iloc[:,:5].mean(axis=1), column_means_2020_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2020_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:8].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        column_means_2021_df = pd.concat([column_means_2021_df.iloc[:,:5].mean(axis=1), column_means_2021_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2021_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:8].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        column_means_2022_df = pd.concat([column_means_2022_df.iloc[:,:5].mean(axis=1), column_means_2022_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2022_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_totday = pd.concat([filtered_floating_population_totday, tot_df])

filtered_floating_population_totday

100%|██████████| 156/156 [00:23<00:00,  6.67it/s]


,연도,평일_유동인구,주말_유동인구,geometry
0,2020,116.320333,98.387917,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2021,91.805833,73.114167,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2022,108.369000,89.299167,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2020,1767.508500,1412.525417,POLYGON ((127.48881291670402 36.64430329023720...
0,2021,2133.972333,1679.542500,POLYGON ((127.48881291670402 36.64430329023720...
...,...,...,...,...
0,2021,165.902500,66.894583,POLYGON ((127.49105137605241 36.63528850233756...
0,2022,217.775500,95.224167,POLYGON ((127.49105137605241 36.63528850233756...
0,2020,54.346333,27.015000,POLYGON ((127.48433936092893 36.63799230366292...
0,2021,45.899667,22.871667,POLYGON ((127.48433936092893 36.63799230366292...


In [15]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2020년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2020 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2020']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2020['geometry'].unique():
    subset = filtered_floating_population_totday_2020[filtered_floating_population_totday_2020['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2020)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2020)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [16]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2021년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2021 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2021']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2021['geometry'].unique():
    subset = filtered_floating_population_totday_2021[filtered_floating_population_totday_2021['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2021)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2021)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [17]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2022년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2022 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2022']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2022['geometry'].unique():
    subset = filtered_floating_population_totday_2022[filtered_floating_population_totday_2022['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2022)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2022)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m